In [ ]:
from huggingface_hub import login
import pandas as pd
import librosa
import datasets
import numpy as np
from datasets import Dataset, Audio
from datasets import load_dataset, load_metric
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer


In [ ]:
# Login to Hugging Face
api_token = 
login(token=api_token, add_to_git_credential=True)

# Load datasets
df = pd.read_csv("")
df_valid = pd.read_csv("")

In [ ]:
def load_audio(file_path):
    signal, sr = librosa.load(file_path, sr=16000)
    signal_array = np.array(signal)
    return {"path": file_path, "array": signal_array, "sampling_rate": sr}

def load_dataset(dataframe):
    dataset = Dataset.from_pandas(dataframe)
    dataset = dataset.map(
        lambda example: {
            "audio": load_audio(example["audio"]),
            "sentence": example["sentence"]
        },
    )
    return dataset

data_dict = datasets.DatasetDict({
    "train": load_dataset(df.iloc[:int(len(df)*1)]),
    "test": load_dataset(df_valid.iloc[:int(len(df_valid)*1)])
})

In [ ]:
from transformers import AutoProcessor, AutoModelForCTC
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-10k-voxpopuli-ft-ro")

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
data_dict = data_dict.map(prepare_dataset, remove_columns=data_dict.column_names["train"], num_proc=4)

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base-10k-voxpopuli-ft-ro",
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

In [ ]:
model.freeze_feature_encoder()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="",
  group_by_length=True,
  per_device_train_batch_size=16,  
  gradient_accumulation_steps=2, 
  evaluation_strategy="steps",
  num_train_epochs=15, 
  fp16=True,
  gradient_checkpointing=True,
  save_steps=2000, 
  eval_steps=500,  
  logging_steps=200,  
  learning_rate=5e-5,  
  weight_decay=0.01,  
  warmup_steps=500,  
  save_total_limit=2,
  load_best_model_at_end=True,  
  metric_for_best_model="wer",
  push_to_hub=True,
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data_dict["train"],
    eval_dataset=data_dict["test"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()